In [94]:
import json
import random
import string
import time
from collections import defaultdict

from helium import *

In [6]:
driver = start_firefox()

In [14]:
write('20', into=TextField(to_right_of='KAZTP.MCX'))

In [95]:
stocks = [
  "KAZTP.MCX",
  "LSNGP.MCX",
  "MDMG.MCX",
  "SIBN.MCX",
  "IRAO.MCX",
  "SNGS.MCX",
  "MSRS.MCX",
  "TATNP.MCX",
  "NVTK.MCX",
  "LKOH.MCX",
  "BANE.MCX",
  "TTLK.MCX",
  "MRKC.MCX",
  "RTKM.MCX",
  "STSBP.MCX",
  "ROSN.MCX",
  "KRSBP.MCX"
]

In [78]:
def make_shares(stocks):
    shares = defaultdict(int)
    remain_share = 100

    stocks_to_randomize = stocks[::]

    while True:
        if not remain_share:
            break

        if not stocks_to_randomize:
            stocks_to_randomize = stocks[::]

        index = random.randint(0, len(stocks_to_randomize)-1)
        stock = stocks_to_randomize.pop(index)
        share = random.randint(0, min(remain_share, 50))
        if share:
            shares[stock] += share
        remain_share -= share

    assert sum(shares.values()) == 100
    return dict(shares)

In [97]:
with open("portfolios.json", encoding='utf-8') as f:
    portfolios = json.load(f)

print(len(portfolios))

59


In [92]:
def make_random_portfolio():
    # Распределение активов
    share_details = S(".col-xl-9 > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)")
    click(share_details)

    # Сбросить
    reset = S("a.font-weight-bold:nth-child(2) > u:nth-child(1)")
    click(reset)

    # Генерим и вводим доли
    shares = make_shares(stocks)
    print(json.dumps(shares))
    for stock, share in shares.items():
        write(str(share), into=TextField(to_right_of=stock))

    # Рассчитать
    doubleclick(S(".btn-primary"))
    try:
        profit = round(float(S("span.text-success:nth-child(3)").web_element.text.replace(',', '.')[:-1]))
    except LookupError:
        profit = -round(float(S("span.text-danger:nth-child(3)").web_element.text.replace(',', '.')[:-1]))
    print(profit)

    portfolio = {"shares": shares, "profit": profit}
    print(json.dumps(portfolio))
    portfolios.append(portfolio)
    with open('portfolios.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(portfolios, indent=2))

In [89]:
len(portfolios)

13

In [93]:
while True:
    make_random_portfolio()
    time.sleep(4)

{"MDMG.MCX": 23, "IRAO.MCX": 16, "LKOH.MCX": 47, "BANE.MCX": 9, "RTKM.MCX": 3, "KRSBP.MCX": 1, "MSRS.MCX": 1}
28
{"shares": {"MDMG.MCX": 23, "IRAO.MCX": 16, "LKOH.MCX": 47, "BANE.MCX": 9, "RTKM.MCX": 3, "KRSBP.MCX": 1, "MSRS.MCX": 1}, "profit": 28}
{"SNGS.MCX": 37, "MDMG.MCX": 11, "LKOH.MCX": 27, "SIBN.MCX": 20, "LSNGP.MCX": 1, "ROSN.MCX": 4}
26
{"shares": {"SNGS.MCX": 37, "MDMG.MCX": 11, "LKOH.MCX": 27, "SIBN.MCX": 20, "LSNGP.MCX": 1, "ROSN.MCX": 4}, "profit": 26}
{"IRAO.MCX": 23, "KAZTP.MCX": 42, "SIBN.MCX": 23, "KRSBP.MCX": 3, "NVTK.MCX": 8, "LKOH.MCX": 1}
204
{"shares": {"IRAO.MCX": 23, "KAZTP.MCX": 42, "SIBN.MCX": 23, "KRSBP.MCX": 3, "NVTK.MCX": 8, "LKOH.MCX": 1}, "profit": 204}
{"RTKM.MCX": 50, "SIBN.MCX": 15, "LSNGP.MCX": 12, "NVTK.MCX": 11, "MSRS.MCX": 5, "ROSN.MCX": 7}
18
{"shares": {"RTKM.MCX": 50, "SIBN.MCX": 15, "LSNGP.MCX": 12, "NVTK.MCX": 11, "MSRS.MCX": 5, "ROSN.MCX": 7}, "profit": 18}
{"KAZTP.MCX": 50, "MRKC.MCX": 26, "SNGS.MCX": 17, "RTKM.MCX": 3, "IRAO.MCX": 2, "TATNP

Traceback (most recent call last):
  File "C:\Users\potyk\PycharmProjects\potyk-base\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\potyk\AppData\Local\Temp\ipykernel_10520\5613732.py", line 2, in <module>
    make_random_portfolio()
  File "C:\Users\potyk\AppData\Local\Temp\ipykernel_10520\974211523.py", line 17, in make_random_portfolio
    doubleclick(S(".btn-primary"))
  File "C:\Users\potyk\PycharmProjects\potyk-base\.venv\Lib\site-packages\helium\__init__.py", line 307, in doubleclick
    _get_api_impl().doubleclick_impl(element)
  File "C:\Users\potyk\PycharmProjects\potyk-base\.venv\Lib\site-packages\helium\_impl\__init__.py", line 206, in doubleclick_impl
    self._perform_mouse_action(element, self._doubleclick)
  File "C:\Users\potyk\PycharmProjects\potyk-base\.venv\Lib\site-packages\helium\_impl\__init__.py", line 37, in f_decorated
    result = f(self, *args, **kwargs)

In [101]:
with open('portfolios.csv', 'w', ) as f:
    f.write(",".join([*stocks, 'profit']) + '\n')
    for portfolio in portfolios:

        portfolio_line = ','.join(map(str,[
            portfolio['shares'].get(stock, 0)
            for stock in stocks
        ] + [portfolio['profit']]))
        f.write(portfolio_line + '\n')


In [104]:
print(*stocks[::-1], sep='\t')

KRSBP.MCX	ROSN.MCX	STSBP.MCX	RTKM.MCX	MRKC.MCX	TTLK.MCX	BANE.MCX	LKOH.MCX	NVTK.MCX	TATNP.MCX	MSRS.MCX	SNGS.MCX	IRAO.MCX	SIBN.MCX	MDMG.MCX	LSNGP.MCX	KAZTP.MCX


In [120]:
import string

cells = []

for letter in reversed(string.ascii_uppercase):
    if letter < 'L':
        cell = "$A" + letter + "$2"
        cells.append(cell)

for letter in reversed(string.ascii_uppercase):
    if letter > 'T':
        cells.append(f"${letter}$2")

formula = '=$AL$2'
for letter, cell in zip(string.ascii_uppercase, cells):
    formula += f"+{letter}2*{cell}"

print(formula)



=$AL$2+A2*$AK$2+B2*$AJ$2+C2*$AI$2+D2*$AH$2+E2*$AG$2+F2*$AF$2+G2*$AE$2+H2*$AD$2+I2*$AC$2+J2*$AB$2+K2*$AA$2+L2*$Z$2+M2*$Y$2+N2*$X$2+O2*$W$2+P2*$V$2+Q2*$U$2
